In [1]:
import geopandas as gpd
from shapely.ops import linemerge
# !pip install passiogo

get shuttle lines
drew them on [here](https://www.google.com/maps/d/u/0/edit?hl=en&mid=15vhtrPbe3NuPElObCVDpo5A2L813JAo&ll=41.78667450063453%2C-87.60665579282379&z=15) from [here](https://safety-security.uchicago.edu/en/transportation/shuttle-services). downloaded KML then used [this tool](https://mygeodata.cloud/converter/kml-to-geojson) to covert to geojson rest is below in py

In [9]:
#!pip install kml2geojson
!k2g uchi_shuttle.kml .

In [ ]:
og = gpd.read_file('style.json', geometry='geometry', crs = "EPSG:4326")

shuttle = og[og.geom_type != 'Point']

# take name color and extract whats between praenthesis
shuttle['color'] = shuttle['name'].str.extract(r'\((.*?)\)', expand=False)


#combine lines for the two midway metra names
green_1 = shuttle[shuttle['name'] == 'Midway Metra (GREEN) 1'].geometry.iloc[0]
green_2 = shuttle[shuttle['name'] == 'Midway Metra (GREEN) 2'].geometry.iloc[0]
# merge the two lines
green_line = linemerge([green_1, green_2]) 
#drop the two lines
shuttle = shuttle[shuttle['name'] != 'Midway Metra (GREEN) 1']
shuttle = shuttle[shuttle['name'] != 'Midway Metra (GREEN) 2']

#add the combined line
shuttle = shuttle.append({'name': 'Midway Metra (GREEN)', 'geometry': green_line, 'color':"GREEN"}, ignore_index=True)

shuttle.crs = "EPSG:4326"

shuttle.drop(columns=['styleUrl'], inplace=True)

shuttle.to_file("uchi_shuttle_lines.geojson", driver='GeoJSON')


